In [10]:
import torch
from gensim.models import KeyedVectors
import pandas as pd

from dataset import MyDataset

In [11]:
print(len(pd.read_csv("data/yelp_val_sample.csv")))
dataset = MyDataset("data/yelp_val_sample.csv", KeyedVectors.load("embedding/yelp.wv").vocab)
loader = torch.utils.data.DataLoader(dataset, batch_size=64)

15690


In [7]:
for _ in loader():
    pass

In [12]:
len(loader)

246

In [13]:
15690 // 64

245